In [1]:
import glob
import json
import glob
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from natsort import natsorted
import matplotlib.pyplot as plt
from collections import defaultdict
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

In [4]:

def load_csv(file_name):
    with open(file_name, "rt") as file_handler:
        data_frame = pd.read_csv(file_handler, index_col=None)

    data_frame = data_frame.rename(columns={'Unnamed: 0': 'index', 'Index, Eval': 'reward'})
    return data_frame


def tabulate_events(dpath, type="eval/mean_reward"):
    summary_iterators = [EventAccumulator(os.path.join(dpath, dname)).Reload() for dname in os.listdir(dpath)]

    tags = summary_iterators[0].Tags()['scalars']


    for it in summary_iterators:
        assert it.Tags()['scalars'] == tags

    out = defaultdict(list)
    steps = []

    for tag in tags:
        if tag == type:
            steps = [e.step for e in summary_iterators[0].Scalars(tag)]

            for events in zip(*[acc.Scalars(tag) for acc in summary_iterators]):
                assert len(set(e.step for e in events)) == 1

                out[tag].append([e.value for e in events])

    return out, steps


def to_csv(dpath = "2Ocak_PPOCNN_Bayes_8x8/tensorboard/target1/"):
    dirs = os.listdir(dpath)

    for dir in dirs:
        d, steps = tabulate_events(dpath + dir)
        tags, values = zip(*d.items())
        np_values = np.array(values)
        # stop

        for index, tag in enumerate(tags):
            df = pd.DataFrame(np_values[index], index=steps, columns=["Index, Eval"])
            df.to_csv(dpath + dir + ".csv")

In [5]:
load_csv("log/iter2/progress.csv")

,time_elapsed,clipfrac,approxkl,eprewmean,policy_loss,nupdates,eplenmean,fps,serial_timesteps,policy_entropy,total_timesteps,explained_variance,rapid_loss,value_loss,episodes
0,0.620457,0.0,0.000178,NaN,-0.002626,1,NaN,206,128,1.945707,128,-7.533491e+00,NaN,1.868782e-03,0
1,10.190213,0.0,0.000081,0.000000,-0.001024,10,1000.00,394,1280,1.942726,1280,1.788139e-07,1.948004,1.954021e-05,1
2,27.172774,0.0,0.000144,0.000000,-0.001494,20,1000.00,403,2560,1.919641,2560,0.000000e+00,1.923696,2.561964e-08,2
3,40.184494,0.0,0.000136,0.000000,-0.000473,30,1000.00,380,3840,1.918085,3840,-1.864002e-01,1.944434,1.826717e-04,3
4,71.372916,0.0,0.000351,0.000000,-0.004410,40,1000.00,7,5120,1.920012,5120,0.000000e+00,1.941430,1.538758e-08,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376,63609.043465,0.0,0.000025,0.005736,0.000176,13760,996.96,304,1761280,1.887312,1761280,0.000000e+00,1.933308,1.496898e-12,1766
1377,63662.581872,0.0,0.000129,0.003899,-0.001732,13770,997.89,5,1762560,1.899637,1762560,1.192093e-07,1.941112,1.056701e-13,1768
1378,63691.721450,0.0,0.000067,0.003899,-0.001138,13780,997.89,423,1763840,1.899065,1763840,0.000000e+00,1.963839,5.131513e-15,1769
1379,63720.096745,0.0,0.000040,0.003899,-0.000112,13790,997.89,297,1765120,1.909317,1765120,5.960464e-08,1.948545,1.768893e-08,1770
